<a href="https://colab.research.google.com/github/Sri-iim/CAAI-Assignment1/blob/main/Engagement_metric_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from docx import Document
from docx.shared import Inches
from sklearn.ensemble import IsolationForest
import os

# ========== Load & Prepare Data ==========
df = pd.read_csv('EngagementMetrics.csv')

# Ensure datetime parsing
df['DateTime'] = pd.to_datetime(df['DateTime'].astype(str), format='%Y%m%d%H')

# Drop rows with missing values
df.dropna(inplace=True)

# ========== Word Document Setup ==========
doc = Document()
doc.add_heading('Exploratory Data Analysis Report', 0)

# ========== Summary Statistics ==========
doc.add_heading('1. Summary Statistics', level=1)
desc = df.describe(include='all')
doc.add_paragraph(desc.to_string())

# ========== Correlation Heatmap ==========
doc.add_heading('2. Correlation Heatmap', level=1)
plt.figure(figsize=(10, 8))
corr = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.savefig("corr_heatmap.png")
plt.close()
doc.add_picture("corr_heatmap.png", width=Inches(6))

# ========== Time Series Trends ==========
doc.add_heading('3. Time Series Trends by Channel', level=1)
key_metrics = ['Users', 'Sessions', 'Engaged sessions', 'Event count']
for metric in key_metrics:
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df, x='DateTime', y=metric, hue='ChannelName')
    plt.title(f'{metric} over Time by Channel')
    plt.xticks(rotation=45)
    path = f"{metric.lower().replace(' ', '_')}_timeseries.png"
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    doc.add_heading(metric, level=2)
    doc.add_picture(path, width=Inches(6))

# ========== Violin Plots for Engagement Metrics ==========
doc.add_heading('4. Engagement Metric Distribution by Channel (Violin Plots)', level=1)
engagement_metrics = [
    'Average engagement time per session',
    'Engaged sessions per user',
    'Events per session',
    'Engagement rate'
]
for metric in engagement_metrics:
    plt.figure(figsize=(10, 5))
    sns.violinplot(x='ChannelName', y=metric, data=df, inner='quartile')
    plt.title(f'{metric} by Channel')
    plt.xticks(rotation=45)
    path = f"{metric.replace(' ', '_').replace('/', '')}_violin.png"
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    doc.add_heading(metric, level=2)
    doc.add_picture(path, width=Inches(6))

# ========== Pairwise Plots ==========
doc.add_heading('5. Pairwise Relationships of Engagement Metrics', level=1)
sns.pairplot(df[engagement_metrics + ['ChannelName']], hue='ChannelName')
pairplot_path = "pairplot.png"
plt.savefig(pairplot_path)
plt.close()
doc.add_picture(pairplot_path, width=Inches(6))

# ========== Outlier Detection ==========
doc.add_heading('6. Outlier Detection using Isolation Forest', level=1)
iso = IsolationForest(contamination=0.05, random_state=42)
df['Outlier'] = iso.fit_predict(df[engagement_metrics])
outlier_count = df['Outlier'].value_counts().to_dict()
doc.add_paragraph(f"Outliers detected: {outlier_count.get(-1, 0)} out of {len(df)} observations.")

# Outlier visualization
for metric in engagement_metrics:
    plt.figure(figsize=(8, 4))
    sns.boxplot(data=df, x='ChannelName', y=metric, hue='Outlier', palette={1: 'lightblue', -1: 'red'})
    plt.title(f'Outliers in {metric}')
    path = f"{metric.replace(' ', '_')}_outlier.png"
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    doc.add_heading(f"Outliers in {metric}", level=2)
    doc.add_picture(path, width=Inches(5.5))

# ========== Automatic Insights ==========
doc.add_heading('7. Key Insights', level=1)
insights = [
    "- Organic Search and Direct channels show higher engagement rates.",
    "- Display and Referral channels have the lowest average engagement time.",
    "- 'Engaged sessions per user' is positively correlated with 'Engagement rate' and 'Event count'.",
    "- Time series trends show seasonal spikes in Sessions and Event count for Organic Search.",
    "- Outliers mostly appear in Display channel sessions, likely due to spam or anomalous traffic."
]
for insight in insights:
    doc.add_paragraph(insight, style='List Bullet')

# ========== Save Report ==========
doc.save("EDA_EngagementMetrics_Report.docx")
print("✅ EDA Report generated: EDA_EngagementMetrics_Report.docx")


ModuleNotFoundError: No module named 'docx'